In [1]:
import sys
sys.path.append('/home/thies/repos/BIU/')
import biu
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
plt.rcParams['svg.fonttype'] = 'none'

import seaborn as sns
import scipy

from rdmpy import RDM
RDM.meta(source="repos/UA_isala/metabolomics/code/initial_investigation.ipynb")

R = biu.R()

# Load data

In [119]:
R("""
load('../data/raw/Isala_tt_merged_genus_fixed.RData')
""")
samples = R("Isala.tt.merged_genus$samples")
taxa    = R("Isala.tt.merged_genus$taxa")
abundances = R("Isala.tt.merged_genus$counts")


samples = samples[(samples.high_quality == 1) & (samples.include == 1) & (samples.flow == 'F1')]

abundances = abundances[abundances.sample_id.isin(samples.sample_id)]
abundances = abundances.pivot(index='sample_id', columns='taxon_id', values='count').fillna(0)
abundances = abundances.rename(columns=dict(zip(taxa.taxon_id, taxa.taxon.apply(lambda x: x.replace(' ', '_')))))

samples = samples.set_index('sample_id')

In [120]:
biu.ops.lst.freq(samples['General.Highest.Education'])

for l in set(samples['General.Highest.Education']):
    if l not in ['PhD', 'Secondary education', 'College', 'University']:
        continue
    #fi
    samples['General.Highest.Education.%s' % l.replace(' ','')] = samples['General.Highest.Education'].apply(lambda x: None if pd.isna(x) else x == l)
    print('General.Highest.Education.%s' % l.replace(' ',''))
#efor

General.Highest.Education.Secondaryeducation
General.Highest.Education.PhD
General.Highest.Education.College
General.Highest.Education.University


In [128]:
samples['Menopause'] = samples['Menopause'].apply(lambda x: {1:True, 0:False}.get(x,None))
samples['Reproductive.Contraception.Hormonal.3months'] = samples['Reproductive.Contraception.Hormonal.3months'].apply(lambda x: {1:True, 0:False}.get(x,None))
samples['Health.Medication.Menopause.HRT'] = samples['Health.Medication.Menopause.HRT'].apply(lambda x: {1:True, 0:False}.get(x,None))
samples['Lifestyle.Drugs.Current_smoker'] = samples['Lifestyle.Drugs.Current_smoker'].apply(lambda x: {1:True, 0:False}.get(x,None))

## Prepare mda input

In [129]:
def write_mda_input(features, metadata, formulas, path):
    biu.utils.fs.mkdirp(path)
    with open('%s/input_formulas.txt' % path, 'w') as ofd:
        for f in formulas:
            ofd.write(f + '\n')
        #efor
    #ewith
    
    features.reset_index().to_csv('%s/input_counts.tsv' % path , sep='\t', index=False)
    metadata.reset_index().to_csv('%s/input_meta.tsv' % path , sep='\t', index=False)
    print("wrote")
#edef

In [150]:
variables = [
    "Menopause",
    "Reproductive.Contraception.Hormonal.3months",
    "Health.Medication.Menopause.HRT",
    "Reproductive.Pregnancy.n",
    "General.Highest.Education.Secondaryeducation",
    "General.Highest.Education.PhD",
    "General.Highest.Education.College",
    "General.Highest.Education.University",
    "General.Age",
    "Lifestyle.Drugs.Current_smoker"
]

formulas = ['~' + '+'.join(variables)]

outdir = '/home/thies/repos/UA_isala/menopause/processing/mda/'
write_mda_input(abundances, samples[variables], formulas, outdir)

wrote


In [165]:
samples[variables].dropna().shape

(1330, 10)

# Run MDA

In [134]:
mda_cmd_code = """
taskset -c {cpus} ~/repos/devel/multidiffabundance/MDA/mda \\
    --docker \\
    --{method} \\
    "{counts}" \\
    "{metadata}" \\
    "{formulas}" \\
    "{output}"
"""

def print_runcode(outdir, method, suffix=None):
    
    print(mda_cmd_code.format(cpus="%d-%d" % (0,5),
                              method=method,
                              counts="%s/input_counts.tsv" % outdir,
                              metadata="%s/input_meta.tsv" % outdir,
                              formulas="%s/input_formulas.txt" % outdir,
                              output="%s/output_%s" % (outdir, method if suffix is None else suffix)))
    
#edef

## Beta

In [162]:
print_runcode(outdir, 'beta')


taskset -c 0-5 ~/repos/devel/multidiffabundance/MDA/mda \
    --docker \
    --beta \
    "/home/thies/repos/UA_isala/menopause/processing/mda//input_counts.tsv" \
    "/home/thies/repos/UA_isala/menopause/processing/mda//input_meta.tsv" \
    "/home/thies/repos/UA_isala/menopause/processing/mda//input_formulas.txt" \
    "/home/thies/repos/UA_isala/menopause/processing/mda//output_beta"



In [166]:
Rbeta = pd.read_csv('/mnt/a-samsung-mz7lm480-ssd/thies/UA_isala/menopause/processing/mda/output_beta/results.full.tsv', sep='\t')

In [167]:
Rbeta

,taxa,variable,effectsize,se,stat,pvalue,qvalue.withinformula,qvalue,formula,method,n,freq,comment
0,mda.beta,MenopauseTRUE,0.000741,NaN,2.418922,0.042,0.042,0.042,~MenopauseTRUE + Reproductive.Contraception.Ho...,beta,3192,"0:3083, 1:109",NaN
1,mda.beta,Reproductive.Contraception.Hormonal.3months,0.000421,NaN,1.376025,NaN,NaN,NaN,~MenopauseTRUE + Reproductive.Contraception.Ho...,beta,3192,NaN,NaN
2,mda.beta,Health.Medication.Menopause.HRT,0.000539,NaN,1.759020,NaN,NaN,NaN,~MenopauseTRUE + Reproductive.Contraception.Ho...,beta,3192,NaN,NaN
3,mda.beta,Reproductive.Pregnancy.n,0.007712,NaN,25.189791,NaN,NaN,NaN,~MenopauseTRUE + Reproductive.Contraception.Ho...,beta,3192,NaN,NaN
4,mda.beta,General.Highest.Education.SecondaryeducationTRUE,0.000111,NaN,0.362664,NaN,NaN,NaN,~MenopauseTRUE + Reproductive.Contraception.Ho...,beta,3192,"0:2572, 1:620",NaN
5,mda.beta,General.Highest.Education.PhDTRUE,0.000174,NaN,0.568367,NaN,NaN,NaN,~MenopauseTRUE + Reproductive.Contraception.Ho...,beta,3192,"0:3083, 1:109",NaN
6,mda.beta,General.Highest.Education.CollegeTRUE,0.000139,NaN,0.454570,NaN,NaN,NaN,~MenopauseTRUE + Reproductive.Contraception.Ho...,beta,3192,"0:2145, 1:1047",NaN
7,mda.beta,General.Highest.Education.UniversityTRUE,0.000202,NaN,0.659887,NaN,NaN,NaN,~MenopauseTRUE + Reproductive.Contraception.Ho...,beta,3192,"0:1801, 1:1391",NaN
8,mda.beta,General.Age,0.002332,NaN,7.616105,NaN,NaN,NaN,~MenopauseTRUE + Reproductive.Contraception.Ho...,beta,3192,NaN,NaN
9,mda.beta,Lifestyle.Drugs.Current_smoker,0.000431,NaN,1.407334,NaN,NaN,NaN,~MenopauseTRUE + Reproductive.Contraception.Ho...,beta,3192,NaN,NaN


## Alpha

In [132]:
print_runcode(outdir, 'alpha')


taskset -c 0-5 ~/repos/devel/multidiffabundance/MDA/mda \
    --docker \
    --alpha \
    "/home/thies/repos/UA_isala/menopause/processing/mda//input_counts.tsv" \
    "/home/thies/repos/UA_isala/menopause/processing/mda//input_meta.tsv" \
    "/home/thies/repos/UA_isala/menopause/processing/mda//input_formulas.txt" \
    "/home/thies/repos/UA_isala/menopause/processing/mda//output_alpha"



In [145]:
Ralpha = pd.read_csv('/mnt/a-samsung-mz7lm480-ssd/thies/repos/UA_isala/menopause/processing/mda/output_alpha/results.tsv', sep='\t')
Ralpha

,taxa,variable,effectsize,se,stat,pvalue,qvalue.withinformula,qvalue,formula,method,n,freq,comment
0,mda.alpha.shannon,MenopauseTRUE,0.152202,0.127169,1.196846,0.231582,0.231582,0.231582,~MenopauseTRUE + Reproductive.Contraception.Ho...,alpha,1330,"0:1221, 1:109",NaN


In [88]:
biu.ops.lst.freq(samples['Menopause'])

{1: 3089, 2: 110, -2147483648: 5}

## MDA

In [135]:
print_runcode(outdir, 'lmclr --limma --deseq2 --maaslin2 --aldex2', suffix="mda")


taskset -c 0-5 ~/repos/devel/multidiffabundance/MDA/mda \
    --docker \
    --lmclr --limma --deseq2 --maaslin2 --aldex2 \
    "/home/thies/repos/UA_isala/menopause/processing/mda//input_counts.tsv" \
    "/home/thies/repos/UA_isala/menopause/processing/mda//input_meta.tsv" \
    "/home/thies/repos/UA_isala/menopause/processing/mda//input_formulas.txt" \
    "/home/thies/repos/UA_isala/menopause/processing/mda//output_mda"



In [143]:
Rmda = pd.read_csv('/mnt/a-samsung-mz7lm480-ssd/thies/repos/UA_isala/menopause/processing/mda/output_mda/results.tsv', sep='\t')

In [172]:
Rmda[Rmda.qvalue < 0.05]

,taxa,variable,effectsize,se,stat,pvalue,qvalue.withinformula,qvalue,formula,method,n,freq,comment
90,Staphylococcus,MenopauseTRUE,-1.385257,0.405612,-3.415226,6.372900e-04,3.058992e-02,3.058992e-02,~MenopauseTRUE + Reproductive.Contraception.Ho...,deseq2,1330,"0:1221, 1:109",NaN
103,Aerococcus,MenopauseTRUE,1.631280,0.015030,3.566961,3.735643e-04,2.441170e-03,2.441170e-03,~MenopauseTRUE + Reproductive.Contraception.Ho...,limma,1330,"0:1221, 1:109",NaN
105,Anaerococcus,MenopauseTRUE,2.825566,0.024661,4.494475,7.560936e-06,1.890234e-04,1.890234e-04,~MenopauseTRUE + Reproductive.Contraception.Ho...,limma,1330,"0:1221, 1:109",NaN
107,Atopobium,MenopauseTRUE,1.168295,0.013058,3.729260,1.998645e-04,1.665537e-03,1.665537e-03,~MenopauseTRUE + Reproductive.Contraception.Ho...,limma,1330,"0:1221, 1:109",NaN
108,Bifidobacterium,MenopauseTRUE,3.181079,0.021402,4.245350,2.329214e-05,2.911518e-04,2.911518e-04,~MenopauseTRUE + Reproductive.Contraception.Ho...,limma,1330,"0:1221, 1:109",NaN
110,Corynebacterium,MenopauseTRUE,1.406032,0.020090,2.520636,1.182687e-02,4.927864e-02,4.927864e-02,~MenopauseTRUE + Reproductive.Contraception.Ho...,limma,1330,"0:1221, 1:109",NaN
111,Dialister,MenopauseTRUE,2.367748,0.022407,4.246592,2.316518e-05,2.911518e-04,2.911518e-04,~MenopauseTRUE + Reproductive.Contraception.Ho...,limma,1330,"0:1221, 1:109",NaN
113,Facklamia,MenopauseTRUE,0.953042,0.014509,2.626901,8.712595e-03,3.960270e-02,3.960270e-02,~MenopauseTRUE + Reproductive.Contraception.Ho...,limma,1330,"0:1221, 1:109",NaN
116,Finegoldia,MenopauseTRUE,2.424085,0.024125,4.142367,3.647312e-05,3.647312e-04,3.647312e-04,~MenopauseTRUE + Reproductive.Contraception.Ho...,limma,1330,"0:1221, 1:109",NaN
119,Lactobacillus_crispatus_group,MenopauseTRUE,2.922496,0.024574,3.555153,3.905872e-04,2.441170e-03,2.441170e-03,~MenopauseTRUE + Reproductive.Contraception.Ho...,limma,1330,"0:1221, 1:109",NaN
